In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import schedule
import time
from datetime import datetime

# Function to fetch data
def fetch_data(last_update_time):
    standings_url = "https://fbref.com/en/comps/20/Bundesliga-Stats"
    years = list(range(2024, 2023, -1))
    all_matches = []

    for year in years:
        data = requests.get(standings_url)
        soup = BeautifulSoup(data.text)
        standings_table = soup.select('table.stats_table')[0]

        links = [l.get("href") for l in standings_table.find_all('a')]
        links = [l for l in links if '/squads/' in l]
        team_urls = [f"https://fbref.com{l}" for l in links]

        previous_season = soup.select("a.prev")[0].get("href")
        standings_url = f"https://fbref.com{previous_season}"

        for team_url in team_urls:
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            data = requests.get(team_url)
            Scores_Fixtures = pd.read_html(data.text, match="Scores & Fixtures")[0]
            Scores_Fixtures = Scores_Fixtures[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'GF', 'GA', 'Opponent', 'Poss']]

            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            data = requests.get(team_url)
            Scores_Fixtures = pd.read_html(data.text, match="Scores & Fixtures")[0]
            Scores_Fixtures = Scores_Fixtures[['Date','Time','Comp','Round','Day','Venue','GF','GA','Opponent','Poss']]
            
            soup = BeautifulSoup(data.text)
            links = [l.get("href") for l in soup.find_all('a')]
            links = [l for l in links if l and 'all_comps/shooting/' in l]
            data = requests.get(f"https://fbref.com{links[0]}")
            shooting = pd.read_html(data.text, match="Shooting")[0]
            shooting.columns = shooting.columns.droplevel()
            
            soup = BeautifulSoup(data.text)
            links = [l.get("href") for l in soup.find_all('a')]
            links = [l for l in links if l and 'all_comps/defense/' in l]
            data = requests.get(f"https://fbref.com{links[0]}")
            defense = pd.read_html(data.text, match="Defensive Actions")[0]
            defense.columns = defense.columns.droplevel()
            
            soup = BeautifulSoup(data.text)
            links = soup.find_all('a')
            links = [l.get("href") for l in links]
            links = [l for l in links if l and 'all_comps/keeper/' in l]
            data = requests.get(f"https://fbref.com{links[0]}")
            goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
            goalkeeping.columns = goalkeeping.columns.droplevel()
    
            soup = BeautifulSoup(data.text)
            links = soup.find_all('a')
            links = [l.get("href") for l in links]
            links = [l for l in links if l and 'all_comps/passing/' in l]
            data = requests.get(f"https://fbref.com{links[0]}")
            passing = pd.read_html(data.text, match="Passing")[0]
            upper_level = passing.columns.get_level_values(0)
            new_columns = [f'{prefix}_{col}' for prefix, col in zip(upper_level, passing.columns.get_level_values(1))]
            passing.columns = new_columns
            passing.rename(columns={passing.columns[0]: 'Date'}, inplace=True)
    
            soup = BeautifulSoup(data.text)
            links = soup.find_all('a')
            links = [l.get("href") for l in links]
            links = [l for l in links if l and 'all_comps/possession/' in l]
            data = requests.get(f"https://fbref.com{links[0]}")
            possession = pd.read_html(data.text, match="Possession")[0]
            possession.columns = possession.columns.droplevel()
        
            #try:
            team_data = Scores_Fixtures.merge(shooting[['Date','Sh']], on = ['Date'])
            team_data = team_data.merge(goalkeeping[['Date','Save%']], on = ['Date'])
            team_data = team_data.merge(passing[['Date','Total_Att','Total_Cmp%']], on = ['Date'])
            team_data = team_data.merge(possession[['Date','Att']], on = ['Date'])
            team_data = team_data.merge(defense[['Date','Blocks']], on = ['Date'])
            
            # Filter data based on last update time
            team_data = Scores_Fixtures[Scores_Fixtures['Date'] > last_update_time]


            team_data = team_data[team_data["Comp"] == "Bundesliga"]
            team_data["Season"] = year
            team_data["Team"] = team_name
            all_matches.append(team_data)
            time.sleep(15)

    # Combine all dataframes and save to a CSV file or process further
    result_df = pd.concat(all_matches)
    result_df.to_csv("weekly_data.csv", index=False)

    # Update last update time to the current time
    return datetime.now()

# Initialize last_update_time with a default or saved timestamp
last_update_time = datetime(2023, 1, 1)

# Schedule the fetch_data function to run every week
schedule.every().week.do(fetch_data, last_update_time)

# Run the scheduled jobs
while True:
    schedule.run_pending()
    time.sleep(10)


import time

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [f"https://fbref.com{l.get('href')}" for l in standings_table.find_all('a') if '/squads/' in l.get('href')]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in links:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        
        Scores_Fixtures = pd.read_html(data.text, match="Scores & Fixtures")[0][['Date','Time','Comp','Round','Day','Venue','GF','GA','Opponent','Poss']]
        shooting = pd.read_html(data.text, match="Shooting")[0].droplevel(0, axis=1)
        defense = pd.read_html(data.text, match="Defensive Actions")[0].droplevel(0, axis=1)
        goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0].droplevel(0, axis=1)
        passing = pd.read_html(data.text, match="Passing")[0].droplevel(0, axis=1)
        possession = pd.read_html(data.text, match="Possession")[0].droplevel(0, axis=1)

        team_data = Scores_Fixtures.merge(shooting[['Date','Sh']], on='Date')
        team_data = team_data.merge(goalkeeping[['Date','Save%']], on='Date')
        team_data = team_data.merge(passing.rename(columns={passing.columns[0]: 'Date'}), on='Date')
        team_data = team_data.merge(possession[['Date','Att']], on='Date')
        team_data = team_data.merge(defense[['Date','Blocks']], on='Date')
        
        team_data = team_data[team_data["Comp"] == "Bundesliga"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        
        time.sleep(10)


In [ ]:
len(all_matches)

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df